# Timing Tests of `fastnumbers` Functions Compared to Equivalent Solutions

In order for you to see the benefit of `fastnumbers`, some timings are collected below for comparison to equivalent python implementations. The fastest time is shown in **bold**. For each test, the resulting time show is time it takes to run the function 100,000 times for a given input; depending on the test this is repeated 5 or 10 times and the average is reported. The numbers may change depending on the machine you are on.

Feel free to download this Jupyter Notebook and run the tests yourself to see how `fastnumbers` performs on your machine (it takes about 1-2 minutes total).

Note that the test results are for Python 2.7. The trends are the same for Python 3.x, but the in general Python 3.x is faster at converting strings to integers than Python 2.7 so rather than a 4x speedup you will only see a 2x speedup for integers on Python 3.x compared to Python 2.7.

Additionally, because of the addition of allowing underscores in numbers in Python 3.6, conversions for both the Python built-in and `fastnumbers` are slower than earlier versions.

The timing runner class is implemented below, and this is used in all the tests to perform the actual timing tests in the sections below. In general you can skip this implementation, but of note is the `THINGS_TO_TIME` tuple, which contains the values that are passed to the functions to type the various input types.

In [1]:
from __future__ import print_function, division
import re
import timeit
from IPython.display import Markdown, display, clear_output

class Timer(object):
    """Class to time functions and make pretty tables of the output."""
    
    # This is a list of all the things we will time with an associated label.
    THINGS_TO_TIME = (
        ('not_a_number', 'Non-number String'),
        ('-41053', 'Int String'),
        ('35892482945872302493947939485729', 'Large Int String'),
        ('-41053.543034e34', 'Float String'),
        ('-41053.543028758302e256', 'Large Float String'),
        (-41053, 'Int'),
        (-41053.543028758302e100, 'Float'),
    )

    # Formatting strings.
    FUNCTION_CALL_FMT = '{}({!r})'
    BOLD_FMT = '**{}**'
    
    def __init__(self, title):
        display(Markdown('### ' + title))
        self.functions = []
    
    def add_function(self, func, label, setup='pass'):
        """Add a function to be timed and compared."""
        self.functions.append((func, setup, label))

    def time_functions(self, repeat=5):
        """Time all the given functions against all input then display results."""

        # Collect the function labels to make the header of this table.
        # Show that the units are seconds for each.
        function_labels = [label + ' (ms)' for _, _, label in self.functions]
        
        # Construct the table strings, formatted in Markdown.
        # Store each line as a string element in a list.
        # This portion here is the table header only for now.
        table = Table()
        table.add_header('Input type', *function_labels)
        
        # For each value, time each function and collect the results.
        for value, value_label in self.THINGS_TO_TIME:
            row = []
            for func, setup, _ in self.functions:
                call = self.FUNCTION_CALL_FMT.format(func, value)
                try:
                    row.append(self._timeit(call, setup, repeat))
                except (ValueError, TypeError):
                    # We might send in some invalid input accidentally.
                    # Ignore those inputs.
                    break

            # Only add this row if the for loop quit without break.
            else:
                # Convert to milliseconds
                row = [v * 1000 for v in row]
                # Make the lowest value bold.
                min_indx = min(enumerate(row), key=lambda x: x[1])[0]
                row = [format(v, '.3f') for v in row]
                row[min_indx] = self.BOLD_FMT.format(row[min_indx])
                table.add_row(value_label, *row)

        # Show the results in a table.
        display(Markdown(str(table)))

    @staticmethod
    def mean(x):
        return sum(x) / len(x)
    
    def _timeit(self, call, setup, repeat=5):
        """Perform the actual timing and return a formatted string of the runtime"""
        return self.mean(timeit.repeat(call, setup, number=100000, repeat=repeat))

class Table(list):
    """List of strings that can be made into a Markdown table."""
    def add_row(self, *elements):
        self.append('|'.join(elements))
    def add_header(self, *elements):
        self.add_row(*elements)
        seperators = ['---'] * len(elements)
        seperators = [sep + (':' if i != 0 else '') for i, sep in enumerate(seperators)]
        self.add_row(*seperators)
    def __str__(self):
        return '\n'.join(self)


## Built-in Functions Drop-in Replacement Timing Results
The following timing tests compare the performance of Python's builtin `int` and `float` functions against the implementations from `fastnumbers` for various input types.

In [2]:
timer = Timer('Timing comparison of `int` functions')
timer.add_function('int', 'builtin')
timer.add_function('int', 'fastnumbers', 'from fastnumbers import int')
timer.time_functions(repeat=100)

### Timing comparison of `int` functions

Input type|builtin (ms)|fastnumbers (ms)
---|---:|---:
Int String|35.035|**27.496**
Large Int String|**34.328**|36.199
Int|15.454|**14.863**
Float|36.194|**33.254**

In [3]:
timer = Timer('Timing comparison of `float` functions')
timer.add_function('float', 'builtin')
timer.add_function('float', 'fastnumbers', 'from fastnumbers import float')
timer.time_functions(repeat=100)

### Timing comparison of `float` functions

Input type|builtin (ms)|fastnumbers (ms)
---|---:|---:
Int String|22.514|**19.566**
Large Int String|**58.143**|64.912
Float String|50.557|**23.069**
Large Float String|**90.675**|95.817
Int|24.399|**18.914**
Float|16.212|**14.272**

## Error-Handling Conversion Functions Timing Results
The following timing tests compare the performance of the `fastnumbers` functions that convert input to numeric types while doing error handling with common equivalent pure-Python implementations.

You should note that included in the "try/except" and "regex" timings is about 10-15 ms worth of function call overhead that does not affect the `fastnumbers` functions because they are implemented as C-extensions. To be completely fair, you can mentally subtract about 10 ms from those times to when comparing the `fastnumbers` implementation.

In [4]:
def int_re(x, int_match=re.compile(r'[-+]?\d+$').match):
    """Function to simulate fast_int but with regular expressions."""
    try:
        if int_match(x):
            return int(x)
        else:
            return x
    except TypeError:
        return int(x)

def int_try(x):
    """Function to simulate fast_int but with try/except."""
    try:
        return int(x)
    except ValueError:
        return x

timer = Timer('Timing comparison of `int` functions with error handling')
timer.add_function('int_try', 'try/except', 'from __main__ import int_try')
timer.add_function('int_re', 'regex', 'from __main__ import int_re')
timer.add_function('fast_int', 'fastnumbers', 'from fastnumbers import fast_int')
timer.time_functions()

### Timing comparison of `int` functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|272.911|62.653|**18.500**
Int String|38.884|116.357|**21.452**
Large Int String|48.573|157.858|**38.526**
Float String|282.893|84.613|**18.530**
Large Float String|300.337|86.163|**19.691**
Int|28.510|162.114|**16.468**
Float|48.686|202.016|**33.655**

In [5]:
def float_re(x, float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate fast_float but with regular expressions."""
    try:
        if float_match(x):
            return float(x)
        else:
            return x
    except TypeError:
        return float(x)

def float_try(x):
    """Function to simulate fast_float but with try/except."""
    try:
        return float(x)
    except ValueError:
        return x

timer = Timer('Timing comparison of `float` functions with error handling')
timer.add_function('float_try', 'try/except', 'from __main__ import float_try')
timer.add_function('float_re', 'regex', 'from __main__ import float_re')
timer.add_function('fast_float', 'fastnumbers', 'from fastnumbers import fast_float')
timer.time_functions()

### Timing comparison of `float` functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|172.892|67.697|**16.702**
Int String|35.321|179.219|**34.656**
Large Int String|73.149|241.994|**65.323**
Float String|65.618|225.819|**23.097**
Large Float String|109.239|266.561|**96.861**
Int|31.681|175.462|**21.094**
Float|26.710|172.219|**14.276**

In [6]:
def real_re(x,
            int_match=re.compile(r'[-+]?\d+$').match,
            real_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate fast_real but with regular expressions."""
    try:
        if int_match(x):
            return int(x)
        elif real_match(x):
            return float(x)
        else:
            return x
    except TypeError:
        if type(x) in (float, int):
            return x
        else:
            raise TypeError

def real_try(x):
    """Function to simulate fast_real but with try/except."""
    try:
        a = float(x)
    except ValueError:
        return x
    else:
        b = int(a)
        return b if a == b else b

timer = Timer('Timing comparison of `float` (but coerce to `int` if possible) functions with error handling')
timer.add_function('real_try', 'try/except', 'from __main__ import real_try')
timer.add_function('real_re', 'regex', 'from __main__ import real_re')
timer.add_function('fast_real', 'fastnumbers', 'from fastnumbers import fast_real')
timer.time_functions()

### Timing comparison of `float` (but coerce to `int` if possible) functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|168.473|113.122|**17.018**
Int String|70.070|116.401|**22.137**
Large Int String|141.929|200.529|**70.108**
Float String|235.226|301.649|**40.110**
Large Float String|331.481|530.482|**235.572**
Int|78.729|279.780|**29.545**
Float|155.621|252.015|**101.485**

In [7]:
def forceint_re(x,
                int_match=re.compile(r'[-+]\d+$').match,
                float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate fast_forceint but with regular expressions."""
    try:
        if int_match(x):
            return int(x)
        elif float_match(x):
            return int(float(x))
        else:
            return x
    except TypeError:
        return int(x)

def forceint_try(x):
    """Function to simulate fast_forceint but with try/except."""
    try:
        return int(x)
    except ValueError:
        try:
            return int(float(x))
        except ValueError:
            return x

timer = Timer('Timing comparison of forced `int` functions with error handling')
timer.add_function('forceint_try', 'try/except', 'from __main__ import forceint_try')
timer.add_function('forceint_re', 'regex', 'from __main__ import forceint_re')
timer.add_function('fast_forceint', 'fastnumbers', 'from fastnumbers import fast_forceint')
timer.time_functions()

### Timing comparison of forced `int` functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|617.751|122.645|**17.852**
Int String|40.844|114.356|**25.160**
Large Int String|73.109|382.890|**56.771**
Float String|600.087|503.764|**50.060**
Large Float String|575.366|478.887|**145.765**
Int|28.710|162.589|**14.469**
Float|50.524|201.221|**31.098**

## Checking Functions Timing Results
The following timing tests compare the performance of the `fastnumbers` functions that check if an input *could* be converted to numeric type with common equivalent pure-Python implementations.

In [8]:
def isint_re(x, int_match=re.compile(r'[-+]?\d+$').match):
    """Function to simulate isint but with regular expressions."""
    t = type(x)
    return t == int if t in (float, int) else bool(int_match(x))

def isint_try(x):
    """Function to simulate isint but with try/except."""
    try:
        int(x)
    except ValueError:
        return False
    else:
        return type(x) != float

timer = Timer('Timing comparison to check if value can be converted to `int`')
timer.add_function('isint_try', 'try/except', 'from __main__ import isint_try')
timer.add_function('isint_re', 'regex', 'from __main__ import isint_re')
timer.add_function('isint', 'fastnumbers', 'from fastnumbers import isint')
timer.time_functions()

### Timing comparison to check if value can be converted to `int`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|306.782|135.393|**17.263**
Int String|57.389|149.888|**17.908**
Large Int String|77.451|171.037|**21.803**
Float String|277.937|132.323|**16.654**
Large Float String|289.279|170.476|**18.808**
Int|52.952|49.553|**13.507**
Float|66.003|44.426|**13.614**

In [9]:
def isfloat_re(x, float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate isfloat but with regular expressions."""
    t = type(x)
    return t == float if t in (float, int) else bool(float_match(x))

def isfloat_try(x):
    """Function to simulate isfloat but with try/except."""
    try:
        float(x)
    except ValueError:
        return False
    else:
        return type(x) != int

timer = Timer('Timing comparison to check if value can be converted to `float`')
timer.add_function('isfloat_try', 'try/except', 'from __main__ import isfloat_try')
timer.add_function('isfloat_re', 'regex', 'from __main__ import isfloat_re')
timer.add_function('isfloat', 'fastnumbers', 'from fastnumbers import isfloat')
timer.time_functions()

### Timing comparison to check if value can be converted to `float`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|189.788|139.454|**18.397**
Int String|66.991|147.784|**18.194**
Large Int String|96.041|182.552|**22.747**
Float String|103.399|206.448|**21.206**
Large Float String|143.771|230.017|**20.679**
Int|52.748|54.796|**16.458**
Float|51.756|54.838|**23.877**

In [10]:
def isreal_re(x, real_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate isreal but with regular expressions."""
    return type(x) in (float, int) or bool(real_match(x))

def isreal_try(x):
    """Function to simulate isreal but with try/except."""
    try:
        float(x)
    except ValueError:
        return False
    else:
        return True

timer = Timer('Timing comparison to check if value can be converted to `float` or `int`')
timer.add_function('isreal_try', 'try/except', 'from __main__ import isreal_try')
timer.add_function('isreal_re', 'regex', 'from __main__ import isreal_re')
timer.add_function('isreal', 'fastnumbers', 'from fastnumbers import isreal')
timer.time_functions()

### Timing comparison to check if value can be converted to `float` or `int`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|202.218|203.930|**26.298**
Int String|63.322|264.557|**30.735**
Large Int String|101.732|254.111|**31.093**
Float String|109.142|237.773|**22.144**
Large Float String|121.394|244.170|**33.795**
Int|35.592|50.072|**27.803**
Float|34.468|37.155|**14.703**

In [11]:
def isintlike_re(x,
                 int_match=re.compile(r'[-+]?\d+$').match,
                 float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate isintlike but with regular expressions."""
    try:
        if int_match(x):
            return True
        elif float_match(x):
            return float(x).is_integer()
        else:
            return False
    except TypeError:
        return int(x) == x

def isintlike_try(x):
    """Function to simulate isintlike but with try/except."""
    try:
        a = int(x)
    except ValueError:
        try:
            a = float(x)
        except ValueError:
            return False
        else:
            return a.is_integer()
    else:
        return a == float(x)

timer = Timer('Timing comparison to check if value can be coerced losslessly to `int`')
timer.add_function('isintlike_try', 'try/except', 'from __main__ import isintlike_try')
timer.add_function('isintlike_re', 'regex', 'from __main__ import isintlike_re')
timer.add_function('isintlike', 'fastnumbers', 'from fastnumbers import isintlike')
timer.time_functions()

### Timing comparison to check if value can be coerced losslessly to `int`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|625.654|160.786|**18.643**
Int String|89.442|77.889|**16.650**
Large Int String|146.634|110.638|**21.346**
Float String|424.028|363.714|**19.040**
Large Float String|475.162|500.645|**30.513**
Int|125.590|207.581|**16.015**
Float|150.396|302.512|**52.146**